given a list of user-track find the number of total replays and total songs listened per user 

In [1]:
import pyspark
from random import randint, random, choice
import matplotlib.pyplot as plt

sc = pyspark.SparkContext('local[*]')

In [3]:
from dataGenerator.rowGenerator import *
from dataGenerator.datasetGenerator import *
from datetime import datetime

dataset = datasetGenerator()
dataset.addGenerator(idGenerator(),prefix="user",min=1,max=100,keyName="userId")
dataset.addGenerator(idGenerator(),prefix="track",min=1,max=5000,keyName="trackId")

rdd = sc.parallelize(dataset.generateDataset(500000)).persist()
rdd.take(5)

[{'userId': 'user_0058', 'trackId': 'track_00794'},
 {'userId': 'user_0022', 'trackId': 'track_04088'},
 {'userId': 'user_0022', 'trackId': 'track_01762'},
 {'userId': 'user_0033', 'trackId': 'track_00785'},
 {'userId': 'user_0087', 'trackId': 'track_00768'}]

In [4]:
# pattern count occurrences da joinovertimedataset
# count occurrences of each user-track
rdd.map(lambda x: ((x.get("userId"),x.get("trackId")),1)).reduceByKey(lambda a,b: a+b).top(5, lambda x: x[1])

[(('user_0044', 'track_02782'), 8),
 (('user_0030', 'track_01310'), 8),
 (('user_0051', 'track_02085'), 7),
 (('user_0100', 'track_02387'), 7),
 (('user_0003', 'track_00292'), 7)]

In [5]:
# non serve il distinct perche' STO CONTANDO LE OCCURRENCES DI (USER,TRACK), NON SOLO USER:
rdd.map(lambda x: (x.get("userId"),x.get("trackId"))).distinct().map(lambda x: ((x[0],x[1]),1)).reduceByKey(lambda a,b: a+b).top(5, lambda x: x[1])

[(('user_0040', 'track_02350'), 1),
 (('user_0024', 'track_01038'), 1),
 (('user_0032', 'track_03440'), 1),
 (('user_0084', 'track_00537'), 1),
 (('user_0012', 'track_03742'), 1)]

In [6]:
# how many replays per user
rdd.map(lambda x: (x.get("userId"), 1)).reduceByKey(lambda a,b: a+b).top(5, lambda x: x[1])

[('user_0041', 5272),
 ('user_0061', 5192),
 ('user_0090', 5141),
 ('user_0032', 5134),
 ('user_0020', 5106)]

In [7]:
# how many songs per user COUNT OCCURRENCES DI SOLO USER QUINDI HA BISOGNO DI DISTINCT
rdd.map(lambda x: (x.get("userId"), x.get("trackId"))).distinct().map(lambda x: (x[0],1)).reduceByKey(lambda a,b: a+b).top(5, lambda x: x[1])

[('user_0041', 3242),
 ('user_0032', 3232),
 ('user_0023', 3223),
 ('user_0072', 3216),
 ('user_0003', 3213)]

In [27]:
# how many replays per song
songreplays = rdd.map(lambda x: (x.get("trackId"), 1)).reduceByKey(lambda a,b: a+b).persist()
songreplays.top(10, lambda x: x[1])

[('track_00607', 134),
 ('track_04041', 133),
 ('track_00038', 132),
 ('track_01386', 132),
 ('track_03399', 132),
 ('track_01297', 132),
 ('track_03761', 131),
 ('track_02842', 131),
 ('track_03254', 130),
 ('track_04322', 129)]

In [45]:
# on average how many replays per song
rdd.map(lambda x: ((x.get("trackId"), x.get("userId")),1)).reduceByKey(lambda a,b: a+b)\
        .map(lambda x: (x[0][0],(x[1],1)))\
        .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))\
        .map(lambda x: (x[0], x[1][0]/x[1][1])).top(10, lambda x: x[0])

[('track_05000', 1.6935483870967742),
 ('track_04999', 1.4393939393939394),
 ('track_04998', 1.6),
 ('track_04997', 1.507936507936508),
 ('track_04996', 1.5737704918032787),
 ('track_04995', 1.4912280701754386),
 ('track_04994', 1.53125),
 ('track_04993', 1.5614035087719298),
 ('track_04992', 1.576271186440678),
 ('track_04991', 1.6774193548387097)]